Batch run contoso-chat

In [1]:
from promptflow import PFClient
from promptflow.entities import Run

pf_client = PFClient()

In [2]:
# load flow
flow = "../contoso-chat"
# load data
data = "../data/salestestdata.jsonl"

In [3]:
# get current time stamp for run name
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%Y_%m_%d_%H%M%S")
run_name = timestamp+"chat_base_run"
print(run_name)

2024_06_04_143107chat_base_run


In [4]:
# Get a pf client to manage runs
pf = PFClient()

# Initialize an Run object
base_run = Run(
    flow=flow,
    # run flow against local data or existing run, only one of data & run can be specified.
    data=data,
    #run="<existing-run-name>",
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
    },
    #variant="${summarize_text_content.variant_0}"
)

# Create the run
result = pf.runs.create_or_update(base_run)

print(result)

Starting prompt flow service...
Start prompt flow service on port 23333, version: 1.11.0.


[2024-06-04 14:31:11 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run contoso_chat_variant_0_20240604_143107_402558, log path: /home/vscode/.promptflow/.runs/contoso_chat_variant_0_20240604_143107_402558/logs.txt


You can stop the prompt flow service with the following command:'pf service stop'.
Alternatively, if no requests are made within 1 hours, it will automatically stop.
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=contoso_chat_variant_0_20240604_143107_402558
2024-06-04 14:31:25 +0000    3330 execution.bulk     INFO     Process 3352 terminated.
2024-06-04 14:31:25 +0000    3330 execution.bulk     WARNING  Process 3368 had been terminated.
2024-06-04 14:31:25 +0000    3330 execution.bulk     WARNING  Process 3362 had been terminated.
2024-06-04 14:31:25 +0000    3330 execution.bulk     WARNING  Process 3372 had been terminated.
name: contoso_chat_variant_0_20240604_143107_402558
created_on: '2024-06-04T14:31:07.387086'
status: Completed
display_name: contoso_chat_variant_0_20240604_143107_402558
description:
tags:
properties:
  flow_path: /workspaces/contoso-chat-backend/contoso-chat
  output_path: /home/vscode/.promptflow/.runs/contoso_chat_variant_0_2

In [5]:
result_df = pf_client.get_details(base_run.name, all_results=True)

In [6]:
result_df.head(10)

,inputs.customerId,inputs.question,inputs.line_number,inputs.chat_history,outputs.answer,outputs.context
0,4,tell me about your hiking jackets,0,[],Hey Sarah Lee! 😊 Let me tell you about our hik...,"[{'id': '17', 'title': 'RainGuard Hiking Jacke..."
1,1,Do you have any climbing gear?,1,[],"Hey John! 😄 Absolutely, we have some fantastic...","[{'id': '9', 'title': 'SummitClimber Backpack'..."
2,3,Can you tell me about your selection of tents?,2,[],"Hey Michael! 🏕️ When it comes to tents, we've ...","[{'id': '15', 'title': 'SkyView 2-Person Tent'..."
3,6,Do you have any hiking boots?,3,[],"Hey Emily! 👋 Absolutely, we have some awesome ...","[{'id': '4', 'title': 'TrekReady Hiking Boots'..."
4,2,What gear do you recommend for hiking?,4,[],"Hey Jane Doe! 👋 For hiking, I recommend the Tr...","[{'id': '10', 'title': 'TrailBlaze Hiking Pant..."


Run contoso-chat evaluation

In [7]:
import os
from azure.ai.generative.evaluate import evaluate
import json
import numpy as np
from azure.ai.resources.client import AIClient
from azure.identity import InteractiveBrowserCredential, DefaultAzureCredential
from azure.ai.resources.entities import AzureOpenAIModelConfiguration
from azure.ai.generative.evaluate.metrics import PromptMetric
import nest_asyncio
nest_asyncio.apply()

ModuleNotFoundError: No module named 'azure.ai.generative'

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()


In [ ]:
config_path = "../config.json"

In [ ]:

#custom_prompt_metric = PromptMetric.from_template(path="metrics/my_custom_relevance.jinja2", name="my_custom_relevance")
client = AIClient.from_config(credential=credential, path=config_path)
tracking_uri = client.tracking_uri

In [ ]:
connection = client.connections.get("aoai-connection")

config = AzureOpenAIModelConfiguration.from_connection(connection, model_name="gpt-4",
                                                           deployment_name="gpt-4")

In [ ]:
result = evaluate(  # This will log metric/artifacts using mlflow
    evaluation_name="contoso-chat-sales-only-eval",
    data=result_df,
    task_type="qa",
    #metrics_list=[custom_prompt_metric, "gpt_groundedness", answer_length],
    metrics_list=["gpt_groundedness", "gpt_relevance", "gpt_coherence", "gpt_fluency"],
    model_config=config,
    data_mapping={
        # reference data
        "customerId": "${inputs.customerId}",
        "question": "${inputs.question}",
        "contexts": "${outputs.context}",
        # reference the run's output
        "answer": "${outputs.answer}",
    },
    tracking_uri=tracking_uri,
    output_path=os.getcwd() + "/downloaded_artifacts/remote"
)
print(result)
print(result.metrics_summary)
print("Studio Url")
print(result.studio_url)    